In [ ]:
# ============================================================================== #
#             Binary Constrained PSO (BCPSO) - Statistical Analysis              #
# ============================================================================== #


# === Core Imports and Setup =================================================
import pandas as pd, numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from io import BytesIO
from openpyxl import Workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
import warnings
warnings.filterwarnings('ignore')

# --- ADDITION: Configuration for statistical runs ---
NUM_RUNS_PER_BUDGET = 30


# === 0. Upload Excel ==========================================================
print("Please upload your mapped_dataset file.")
uploaded = files.upload()
xlsx_name = next(iter(uploaded))
print(f"Successfully uploaded: {xlsx_name}")


# === 1. Load & reshape ========================================================
print("\nLoading and preparing data...")
df = (
    pd.read_excel(xlsx_name)
      [[ 'tc_id','us_id', 'tc_executiontime', 'us_businessvalue']]
      .rename(columns={
          'tc_id'          : 'test',   # test‑case ID
          'us_id'          : 'req',    # requirement ID
          'tc_executiontime': 'cost',  # cost / effort
          'us_businessvalue': 'bval'   # business value
      })
)

# ----- lookup helpers ---------------------------------------------------------
req_sets = df.groupby('test')['req'].apply(set).to_dict()     # test → {reqs}
costs    = df.groupby('test')['cost'].first().to_dict()       # test → cost
bvalue   = df.groupby('req')['bval'].first().to_dict()        # req  → value

# --- CHANGE: Sort test IDs to ensure a stable, reproducible order. ---
tests         = sorted(list(req_sets.keys()))
n_tests       = len(tests)
all_reqs, k   = set(df['req'].unique()), len(df['req'].unique())

# --- CHANGE: Correctly sum unique test costs to prevent double-counting. ---
# tot_cost      = df['cost'].sum()
tot_cost      = df.drop_duplicates(subset=['test'])['cost'].sum()
tot_bval      = sum(bvalue.values())

# --- ADDITION: Create a lookup map for the stricter "fully covered" metric. ---
# This maps each requirement to the full set of tests needed to cover it.
req_to_tests_map = df.groupby('req')['test'].apply(set).to_dict()
print(f"Data prepared: {n_tests} Test Cases, {k} Requirements.")
print(f"Total possible cost (100% budget): {tot_cost:.2f}\n")


def evaluate_solution(sol):
    sel_tests = [tests[i] for i, bit in enumerate(sol) if bit]
    cov_reqs  = set().union(*(req_sets.get(t, set()) for t in sel_tests)) if sel_tests else set()
    cov_pct   = 100 * len(cov_reqs) / k
    cost_val  = sum(costs.get(t, 0) for t in sel_tests)
    # Note: bvc_pct is based on atomic coverage and not used in final analysis.
    bvc_pct   = 100 * sum(bvalue.get(r, 0) for r in cov_reqs) / tot_bval if tot_bval > 0 else 0
    return cov_pct, cost_val, bvc_pct, sel_tests


# === 2. PSO parameters ========================================================
POP_SIZE, C1, C2 = 20, 1.5, 1.5
# --- CHANGE: MAX_EVALS is adjusted for practical runtime across 30 runs. ---
MAX_EVALS        = 30000
W_MAX, W_MIN     = 0.9, 0.4
# --- CHANGE: Penalty is increased for a stronger constraint. ---
# PENALTY          = 100
PENALTY          = 1000

# --- ADDITION: A more robust fitness function that penalizes proportionally. ---
def fitness_fn(cov, cost, budget):
    if cost <= budget:
        return cov
    excess = cost - budget
    return cov - PENALTY * (1 + excess / budget)


def run_pso(budget):
    positions  = np.random.randint(0, 2, (POP_SIZE, n_tests))
    velocities = np.random.uniform(0, 1, (POP_SIZE, n_tests))
    pbest_pos, pbest_fit = positions.copy(), np.full(POP_SIZE, -np.inf)
    evals, gbest_fit, gbest_pos = 0, -np.inf, None

    # --- ADDITION: "Warm Start" seeding to improve convergence and stability. ---
    affordable_tests = {t: costs[t] for t in tests if costs.get(t, 0) <= budget and costs.get(t, 0) > 0}
    if affordable_tests:
        efficiency = {t: len(req_sets.get(t, set())) / cost for t, cost in affordable_tests.items()}
        sorted_by_efficiency = sorted(efficiency.keys(), key=lambda t: efficiency[t], reverse=True)
        greedy_seed_tests = set(); current_cost = 0
        for test_id in sorted_by_efficiency:
            if current_cost + costs[test_id] <= budget:
                greedy_seed_tests.add(test_id); current_cost += costs[test_id]
        base_seed_vector = np.zeros(n_tests, dtype=int)
        for test_id in greedy_seed_tests: base_seed_vector[tests.index(test_id)] = 1
        for i in range(POP_SIZE):
            mutated_seed = base_seed_vector.copy()
            mutation_indices = np.random.choice(n_tests, size=5, replace=False)
            mutated_seed[mutation_indices] = 1 - mutated_seed[mutation_indices]
            positions[i] = mutated_seed
        positions[0] = base_seed_vector

    # initial evaluation -------------------------------------------------------
    for i in range(POP_SIZE):
        cov, cost, _, _ = evaluate_solution(positions[i])
        # --- CHANGE: Use the more robust fitness function. ---
        # fit = cov - PENALTY if cost > budget else cov
        fit = fitness_fn(cov, cost, budget)
        pbest_fit[i]    = fit
        # --- CHANGE: Global best must be a feasible solution. ---
        if cost <= budget and fit > gbest_fit:
            gbest_fit, gbest_pos = fit, positions[i].copy()
        evals          += 1

    neigh = {i: [(i-1) % POP_SIZE, (i+1) % POP_SIZE] for i in range(POP_SIZE)}

    # PSO loop -----------------------------------------------------------------
    while evals < MAX_EVALS:
        w = W_MAX - (W_MAX - W_MIN) * (evals / MAX_EVALS)
        for i in range(POP_SIZE):
            lbest = pbest_pos[max(neigh[i] + [i], key=pbest_fit.__getitem__)]
            r1, r2 = np.random.rand(n_tests), np.random.rand(n_tests)
            velocities[i] = (w*velocities[i]+C1*r1*(pbest_pos[i]-positions[i])+C2*r2*(lbest-positions[i]))
            sig          = 1 / (1 + np.exp(-velocities[i]))
            positions[i] = (np.random.rand(n_tests) < sig).astype(int)
            cov, cost, _, _ = evaluate_solution(positions[i])
            # --- CHANGE: Use the more robust fitness function. ---
            # fit = cov - PENALTY if cost > budget else cov
            fit = fitness_fn(cov, cost, budget)
            evals          += 1
            if fit > pbest_fit[i]:
                pbest_fit[i], pbest_pos[i] = fit, positions[i].copy()
            # --- CHANGE: Global best must be a feasible solution. ---
            # if fit > gbest_fit:
            #     gbest_fit, gbest_pos = fit, positions[i].copy()
            if cost <= budget and fit > gbest_fit:
                gbest_fit, gbest_pos = fit, positions[i].copy()
        if evals >= MAX_EVALS: break

    if gbest_pos is None: return -np.inf, 0, 0, 0, []
    cov, cost, bvc, sel_tests = evaluate_solution(gbest_pos)
    return cov, cov, bvc, cost, sel_tests

# === 3. Run budgets & stream results =========================================
all_runs_data = []
print(f"--- Starting Statistical Run ({NUM_RUNS_PER_BUDGET} runs per budget) ---")

for pct in range(5, 100, 5):
    budget = pct / 100 * tot_cost
    print(f"\n--- Running for Budget: {pct}% (Max Time: {budget:.2f}) ---")
    for run in range(NUM_RUNS_PER_BUDGET):
        fit, cov, bvc, cost_val, sel = run_pso(budget)

        print(f"Run {run+1:>2}: Budget={pct:>.1f} | Fitness={fit:7.2f} | ReqCov%={cov:8.2f} | Cost={cost_val:9.2f}", flush=True)

        all_runs_data.append({
            'Time Budget %': pct, 'Run': run + 1,
            'Selected Tests': sel
        })

# === 4. Analysis and Plotting ===================================
print("\n\n" + "="*60 + "\n--- Post-Run Statistical Analysis and Visualization ---\n" + "="*60)
raw_results_df = pd.DataFrame(all_runs_data)

def calculate_final_metrics(selected_tests, req_sets_map, bval_map, req_to_tests, total_reqs):
    selected_tests_set = set(selected_tests)
    atomic_covered_reqs = set().union(*(req_sets_map.get(t, set()) for t in selected_tests_set))

    atomic_req_cov_pct = (len(atomic_covered_reqs) / total_reqs) * 100
    total_atomic_bv = sum(bval_map.get(r, 0) for r in atomic_covered_reqs)

    fully_covered_bv, fully_covered_req_count = 0, 0
    for req_id, required_tests in req_to_tests.items():
        if required_tests.issubset(selected_tests_set):
            fully_covered_bv += bval_map.get(req_id, 0)
            fully_covered_req_count += 1

    fully_covered_req_cov_pct = (fully_covered_req_count / total_reqs) * 100

    return atomic_req_cov_pct, total_atomic_bv, fully_covered_req_cov_pct, fully_covered_bv

all_metrics = raw_results_df['Selected Tests'].apply(lambda tests: calculate_final_metrics(tests, req_sets, bvalue, req_to_tests_map, k))
raw_results_df['Atomic Req Coverage %'] = all_metrics.apply(lambda x: x[0])
raw_results_df['Total Atomic BV'] = all_metrics.apply(lambda x: x[1])
raw_results_df['Fully Covered Req %'] = all_metrics.apply(lambda x: x[2])
raw_results_df['Fully Covered BV'] = all_metrics.apply(lambda x: x[3])
raw_results_df['# of Tests'] = raw_results_df['Selected Tests'].apply(len)
raw_results_df['Cost'] = raw_results_df['Selected Tests'].apply(lambda sel: sum(costs.get(t, 0) for t in sel))

summary_df = raw_results_df.groupby('Time Budget %').agg(
    mean_atomic_req_cov=('Atomic Req Coverage %', 'mean'), median_atomic_req_cov=('Atomic Req Coverage %', 'median'),
    mean_fully_covered_req_cov=('Fully Covered Req %', 'mean'), median_fully_covered_req_cov=('Fully Covered Req %', 'median'),
    mean_atomic_bv=('Total Atomic BV', 'mean'), median_atomic_bv=('Total Atomic BV', 'median'),
    mean_fully_covered_bv=('Fully Covered BV', 'mean'), median_fully_covered_bv=('Fully Covered BV', 'median'),
).reset_index()

print("\n--- Final Statistical Summary Table ---")
print(summary_df.to_string())

# --- ADDITION: Plotting section updated to include Mean and Median plots ---
fig, axes = plt.subplots(3, 2, figsize=(20, 22))
fig.suptitle(f'BCPSO Statistical Analysis for: {xlsx_name} ({NUM_RUNS_PER_BUDGET} Runs)', fontsize=18)

# Row 1: Atomic (lenient) Metrics
axes[0, 0].plot(summary_df['Time Budget %'], summary_df['median_atomic_req_cov'], marker='o', c='black')
axes[0, 0].set_title('Median Atomic ("Touch") Requirement Coverage', fontsize=14); axes[0, 0].set_ylabel('Coverage (%)'); axes[0, 0].set_ylim(0, 105)

axes[0, 1].plot(summary_df['Time Budget %'], summary_df['median_atomic_bv'], marker='o', c='green')
axes[0, 1].set_title('Median Total Atomic BV Covered', fontsize=14); axes[0, 1].set_ylabel('Business Value (Atomic)')

# Row 2: Fully Covered (stricter) Median Metrics
axes[1, 0].plot(summary_df['Time Budget %'], summary_df['median_fully_covered_req_cov'], marker='o', c='blue')
axes[1, 0].set_title('Median "Fully Covered" Requirement Coverage', fontsize=14); axes[1, 0].set_ylabel('Coverage (%)'); axes[1, 0].set_ylim(0, 105)

axes[1, 1].plot(summary_df['Time Budget %'], summary_df['median_fully_covered_bv'], marker='o', c='purple')
axes[1, 1].set_title('Median "Fully Covered" BV', fontsize=14); axes[1, 1].set_ylabel('Business Value (Fully Covered)')

# Row 3: Fully Covered (stricter) Mean Metrics
axes[2, 0].plot(summary_df['Time Budget %'], summary_df['mean_fully_covered_req_cov'], marker='o', c='darkcyan')
axes[2, 0].set_title('Mean "Fully Covered" Requirement Coverage', fontsize=14); axes[2, 0].set_ylabel('Coverage (%)'); axes[2, 0].set_ylim(0, 105)

axes[2, 1].plot(summary_df['Time Budget %'], summary_df['mean_fully_covered_bv'], marker='o', c='darkred')
axes[2, 1].set_title('Mean "Fully Covered" BV', fontsize=14); axes[2, 1].set_ylabel('Business Value (Fully Covered)')

for ax in axes.flat:
    ax.set_xlim(0, 100); ax.set_xticks(np.arange(5, 100, 5)); ax.tick_params(axis='x', rotation=45, labelsize=10)
    ax.set_xlabel('Time Budget (%)'); ax.grid(axis='y', linestyle=':', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.96]); plot_filename = "BCPSO_statistical_plots.png"
fig.savefig(plot_filename)
print(f"\n\nAll plots saved to '{plot_filename}'")
plt.show()

# === 5. Create Final Excel Report =======================================
output_excel_filename = "BCPSO_Statistical_Analysis_Report.xlsx"
print(f"\nGenerating comprehensive Excel report: '{output_excel_filename}'...")
with pd.ExcelWriter(output_excel_filename, engine='openpyxl') as writer:
    summary_df.to_excel(writer, sheet_name='BCPSO Summary Table', index=False)
    raw_results_df.to_excel(writer, sheet_name='BCPSO Raw Run Data', index=False)
    wb = writer.book; ws = wb.create_sheet(title="Analysis Plots")
    img = OpenpyxlImage(plot_filename); img.anchor = 'A1'; ws.add_image(img)
print(f"Report saved successfully.")

Please upload your mapped_dataset file.


Saving mapped_dataset_1.xlsx to mapped_dataset_1 (1).xlsx
Successfully uploaded: mapped_dataset_1 (1).xlsx

Loading and preparing data...
Data prepared: 160 Test Cases, 496 Requirements.
Total possible cost (100% budget): 1398.62

--- Starting Statistical Run (30 runs per budget) ---

--- Running for Budget: 5% (Max Time: 69.93) ---
Run  1: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  2: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  3: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  4: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  5: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  6: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  7: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  8: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run  9: Budget=5.0 | Fitness=  30.44 | ReqCov%=   30.44 | Cost=    69.91
Run 10: 